In [16]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

import stripe
stripe.api_key = os.getenv('STRIPE_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 0

#### App Development Notes:

What seems to be creating the greatest latency issues?
1. The memory, which was supposed to be a token saving/context management system but may be overkill in early stage parts of the conversation
2. The classifers that are needed to preare the prompt (the prompt itself seems to run pretty fast)

In [17]:
#load in the policies to the vectorstore
loaders = []
docs = []

#create a list with the names of the files to be loaded in the data/policies folder
for file in os.listdir('data/policies'):
    if file.endswith('.txt'):
        loader = TextLoader('data/policies/' + file)
        loaders.append(loader)

for loader in loaders:
    docs.extend(loader.load())

if times_embedding_run < 1:
    text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
    documents = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(openai_api_key=key)
    policies = Chroma.from_documents(documents, embeddings)
    times_embedding_run += 1

"Embeddings generated into policies variable"

Created a chunk of size 408, which is longer than the specified 250
Created a chunk of size 377, which is longer than the specified 250
Created a chunk of size 293, which is longer than the specified 250
Created a chunk of size 298, which is longer than the specified 250
Created a chunk of size 332, which is longer than the specified 250
Created a chunk of size 510, which is longer than the specified 250
Created a chunk of size 514, which is longer than the specified 250
Created a chunk of size 350, which is longer than the specified 250
Created a chunk of size 265, which is longer than the specified 250
Created a chunk of size 458, which is longer than the specified 250
Created a chunk of size 667, which is longer than the specified 250
Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 364, which is longer than the specified 250
Created a chunk of size 441, which is longer than the specified 250
Created a chunk of size 360, which is longer tha

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


'Embeddings generated into policies variable'

In [18]:
location_of_issue = Property('location', None, "The location of the issue", required=True, examples=['kitchen', 'bathroom', 'bedroom', 'proactive'])

special_instructions = Property('special_instructions', None, "Any special instructions for the technician", required=False, examples=['please call before arriving', 'please leave a note on the door'])

square_footage = Property('square_footage', None, "The square footage of the property", required=False, examples=['1000', '2000', '3000'])

pest_id = Property('pest_id', None, "The id number of the pest", required=True, examples=['1', '2', '3'])

issues = NaturalLanguagePool([location_of_issue, special_instructions, pest_id, square_footage], "The issues the customer is having", create_new_properties=True)

In [19]:
chat = ChatOpenAI(openai_api_key=key)

initial_prompt = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases."""

history = DynamicChatHistory(chat, system_message=initial_prompt, additional_context="", initial_history=[SystemMessage(content="Hi! My name is George from Better Termite & Pest Control. How can I help you? :)")])

stage_one = ConversationScriptStage('Take lead of conversation', "Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.", lambda: any(isinstance(message, AIMessage) for message in history.history))

stage_two = ConversationScriptStage('Get contact information', "Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.", lambda: customer_profile.is_complete(), pool=customer_profile, show_missing=True, show_complete=True)

stage_three = ConversationScriptStage('Get customer issue', "Gather details about the customer's issue. This goal is considered complete, once you have the pest ID, location of the issue and the square footage of the property. Make sure to check if the customer is seeing multiple issues. Ask only 1 question per response.", lambda: issues.is_complete(), pool=issues, show_missing=True, show_complete=True)

sales_script = ConversationScript([stage_one, stage_two, stage_three])

agent = Conversation(history, chat, sales_script, customer_profile, issues, policies, log_prompt=True)

In [20]:
agent.respond('Hi name is John Smith and I am interested in pest control')

Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: None
{'pest_id': '0', 'special_instructions': None, 'location': None, 'square_footage': None}
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
 Current goal: Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.Use the following pieces of context to answer the users question. If you don't know the answer, just say that you don't know, don't try to m

In [21]:
agent.respond('Yeah sure, thats fine')

Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: None
{'pest_id': '0', 'special_instructions': None, 'location': None, 'square_footage': None}
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hi John! Thanks for reaching out to Better Termite & Pest Control. I’d be happy to help you with your pest control needs. Before we begin, may I ask you a few questions to better understand your issue?
User: Yeah sure, thats fine
 Current goal: Get the customer's contact information. This goal is considered co

In [22]:
agent.respond('Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878')

Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: None
{'pest_id': '0', 'special_instructions': None, 'location': None, 'square_footage': None}
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hi John! Thanks for reaching out to Better Termite & Pest Control. I’d be happy to help you with your pest control needs. Before we begin, may I ask you a few questions to better understand your issue?
User: Yeah sure, thats fine
Agent: Great, thank you John. First, may I have your phone number so we can easil

In [23]:
agent.respond("Last night, I saw a mouse run across the kitchen and it really startled me. I'm not sure if there could be more. I just want them gone.")

Pests: Mice
Pest_Id: 5
Special_Instructions: Mice seen in the kitchen
Location: Kitchen
Square_Footage: None
{'pest_id': '5', 'special_instructions': 'Mice seen in the kitchen', 'location': 'Kitchen', 'square_footage': None}
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hi John! Thanks for reaching out to Better Termite & Pest Control. I’d be happy to help you with your pest control needs. Before we begin, may I ask you a few questions to better understand your issue?
User: Yeah sure, thats fine
Agent: Great, thank you John. F

In [24]:
agent.respond('Maybe, but I would like to add: my home is 2000 square feet')

Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: 2000
Customer Response: I have seen a lot of spiders in my basement.
Pests: Spiders
Pest_Id: 2
Special_Instructions: Spiders are in the basement
Location: Basement
Square_Footage: None
Customer Response: I have noticed some bed bug bites on my body.
Pests: Bed Bugs
Pest_Id: 9
Special_Instructions: Customer is experiencing bed bug bites
Location: None
Square_Footage: None
{'pest_id': '9', 'special_instructions': 'Customer is experiencing bed bug bites', 'location': None, 'square_footage': None}
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conver

Pests: Rodents
Pest_Id: 4
Special_Instructions: Recommend inspection to determine extent of issue. Mention Castle's 7-Point Touch protection plan.
Location: Kitchen
Square_Footage: 2000
{'pest_id': '4', 'special_instructions': 'Recommend inspection to determine extent of issue. Mention Castle', 'location': 'Kitchen', 'square_footage': '2000'}


TypeError: 'NoneType' object is not iterable

In [12]:
agent.issues('Yep seeing in my kitchen')

TypeError: 'NaturalLanguagePool' object is not callable

In [18]:
agent.respond("It's 2500 square feet")

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, thanks for reaching out to us here at Better Termite & Pest Control. I'm happy to assist you with your pest control needs. Do you mind if I ask you a few questions to get a better understanding of your situation?
User: Yeah sure, thats fine
Agent: Great, thanks! Can you kindly provide me with your phone number, email, street address, city, state, and zip code? That way we can get in touch with you and provide a more accurate estimate.
User: Sure my name is George Schulz. 

In [10]:
agent.customer_profile.data

[first_name: George,
 last_name: Schulz,
 phone_number: 571-431-9531,
 email: georgeschulz33@gmail.com,
 street_address: 123 Elm Way,
 city: Gaithersburg,
 state: MD,
 zip: 20878]

In [15]:
show_chat_log(agent.history)

*Initialization Prompt*: Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
------------------------------------
*User*: Hi name is John Smith and I am interested in pest control
------------------------------------
*Agent*: Hey there John! Thanks for reaching out to Better Termite & Pest Control. I'd love to help you out with your pest control needs. May I ask what type of pest issue you're dealing with?
------------------------------------
*User*: Yeah sure, thats fine
------------------------------------
*Agent*: Great! To help you get started, may I have your phone number please?
------------------------------------
*User*: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878
------------------------------------
*Agent*: Thanks, George! To better assist you, could you please let me know what type of pest issue you're dealing with and where is the location of th

In [19]:
agent.respond('Yes, thats right')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, thanks for reaching out to us here at Better Termite & Pest Control. I'm happy to assist you with your pest control needs. Do you mind if I ask you a few questions to get a better understanding of your situation?
User: Yeah sure, thats fine
Agent: Great, thanks! Can you kindly provide me with your phone number, email, street address, city, state, and zip code? That way we can get in touch with you and provide a more accurate estimate.
User: Sure my name is George Schulz. 

In [20]:
agent.respond('Im not sure')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, thanks for reaching out to us here at Better Termite & Pest Control. I'm happy to assist you with your pest control needs. Do you mind if I ask you a few questions to get a better understanding of your situation?
User: Yeah sure, thats fine
Agent: Great, thanks! Can you kindly provide me with your phone number, email, street address, city, state, and zip code? That way we can get in touch with you and provide a more accurate estimate.
User: Sure my name is George Schulz. 

In [21]:
agent.issues.data

[location: Kitchen, around the house,
 special_instructions: Ants are in the kitchen, spiders are around the house,
 pest_id: None,
 square_footage: 2500,
 pests: Ants, spiders]

In [23]:
print(agent.history.get_transcript())


User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, thanks for reaching out to us here at Better Termite & Pest Control. I'm happy to assist you with your pest control needs. Do you mind if I ask you a few questions to get a better understanding of your situation?
User: Yeah sure, thats fine
Agent: Great, thanks! Can you kindly provide me with your phone number, email, street address, city, state, and zip code? That way we can get in touch with you and provide a more accurate estimate.
User: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878
Agent: Thanks for providing that information, George! Can you let me know what type of pests you are having an issue with? That way we can provide the proper treatment.
User: I've been seeing ants in my kitchen and some spiders around the house. Im just fed up!
Agent: I'm sorry to hear that, George! Just to clarify, are you e

In [39]:
import datetime
import random

#a class for mocking a connection to pestpac database for testing purposes of the LLM
class PestPacConnection:
    def __init__(self, api_key, username, password):
        self.api_key = api_key
        self.username = username
        self.password = password
    
    #make sure the default start date is tomorrow's date and end date is tomorrows + 10
    def best_fit(self, lat, long, service, s_date=None, e_date=None, dow=None, time_rg=None):
        if s_date == None:
            s_date = datetime.date.today() + datetime.timedelta(days=1)
        if e_date == None:
            e_date = datetime.date.today() + datetime.timedelta(days=11)
        
        #generate 3 random dates between the start and end date and choose a random tech
        appointments = []
        techs = ['John Smith', 'Jayden Nicholas', 'Gregory Stepheson']
        time_ranges = ['9:00 AM - 12:00 PM', '12:00 PM - 3:00 PM', '3:00 PM - 6:00 PM']
        for i in range(3):
            #get 3 random dates between the start and end date
            date = random.randint(s_date.toordinal(), e_date.toordinal())
            date = datetime.date.fromordinal(date)
            tech = random.choice(techs)
            time = random.choice(time_ranges)
            appointments.append({'date': date, 'tech': tech, 'time': time})

        return appointments
    
    def create_service(self, loc_id, serv, bill_freq, bill_inf, recur_price, recur_time, tech, in_date, instr, in_time, initial_price):
        print('Creating new service')
        print(f' loc_id: {loc_id},\n serv: {serv},\n bill_freq: {bill_freq},\n bill_inf: {bill_inf},\n recur_price: {recur_price},\n recur_time: {recur_time},\n tech: {tech},\n in_date: {in_date},\n instr: {instr},\n in_time: {in_time},\n initial_price: {initial_price}')
        return True

    def create_loc(self, name, address, city, state, zip, phone, email):
        print('Creating new location')
        print(f' name: {name},\n address: {address},\n city: {city},\n state: {state},\n zip: {zip},\n phone: {phone},\n email: {email},\n')
        return True

pp = PestPacConnection(api_key='123', username='123', password='123')
pp.create_loc('George Schulz', '123 Elm Way', 'Gaithersburg', 'MD', '20878', '571-431-9531', 'georgeschulz33@gmail.com')

Creating new location
 name: George Schulz,
 address: 123 Elm Way,
 city: Gaithersburg,
 state: MD,
 zip: 20878,
 phone: 571-431-9531,
 email: georgeschulz33@gmail.com,



True

In [13]:
class Payments:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def get_product(self, program_name):
        programs = {
            'All in One': 'prod_MLignVDWxCwv98',
            'lawnshield': 'prod_NX8oqOKLA2tPIn',
            'castle': 'prod_NX8pSb35sDjLqE'        
        }

        return programs[program_name]
    
    def create_payment_link(self, service_name, initial):
        stripe.api_key = self.api_key
        service_id = self.get_product(service_name)
        unit_amount = int(initial * 100)
        #create the price for the service in the stripe dashboard
        price = stripe.Price.create(currency="usd", unit_amount=unit_amount, product=service_id)
        price_id = price.id
        #create the payment link
        link = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price': price_id,
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success',
            cancel_url='https://example.com/cancel',
        )
        return link.url


pay = Payments(api_key=os.getenv('STRIPE_KEY'))

In [15]:
pay.create_payment_link('castle', 84.99)

'https://checkout.stripe.com/c/pay/cs_test_a12BomIMePybf7XcJHkYeLnvvfFplRLN7i7cE5UtLZuGG7qJT27APCm9D1#fidkdWxOYHwnPyd1blpxYHZxWjA0TTJoRHZOaGtxTUlPdF9PR1VGYGdOXHMwVU4zNFVkXERqbWlITX82VHdmaVxtXFZSYHdOaGZTa0RUa1Y0ZEpjVF9CVV9KYzBnTEB9a1cxUmc0TGNwZEwyNTU2UTVmc1JgZicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl'

In [ ]:
class Knowledge:
    def __init__(self):
        pass

    def find_best_service(self, pest, location):
        #find the best service for the pest and location
        return 'All in One'
    
    def assess_situation(self, pest, location):
        #find the best service for the pest and location
        return 'All in One'
    
    def get_price(self, service):
        #get the price for the service
        return 84.99
    
    